In [ ]:
pip install python-terrier

     |████████████████████████████████| 97 kB 2.9 MB/s 
     |████████████████████████████████| 1.1 MB 36.5 MB/s 
     |████████████████████████████████| 69 kB 7.0 MB/s 
     |████████████████████████████████| 255 kB 26.4 MB/s 
     |████████████████████████████████| 41 kB 207 kB/s 
     |████████████████████████████████| 1.2 MB 42.8 MB/s 
     |████████████████████████████████| 126 kB 47.2 MB/s 
     |████████████████████████████████| 6.4 MB 42.9 MB/s 
     |████████████████████████████████| 596 kB 62.3 MB/s 
     |████████████████████████████████| 294 kB 34.4 MB/s 
     |████████████████████████████████| 72 kB 1.2 MB/s 
     |████████████████████████████████| 291 kB 66.5 MB/s 
     |████████████████████████████████| 59 kB 6.0 MB/s 
     |████████████████████████████████| 45 kB 2.4 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.0-py3-none-any.whl size=103883 sha256=8bfa464a7312396e816dd2276ccc1d72424ea9e30361ed8192f86b682742eb51
  Stored in directory: /root/.cac

In [ ]:
import pyterrier as pt
import lightgbm as lgb
import numpy as np

In [ ]:
if not pt.started():
    pt.init(mem=20000)

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
# 1. identifying a candidate set of documents for each query
print("collecting dataset")
msmarco_dataset = pt.get_dataset("trec-deep-learning-docs")

collecting dataset


In [ ]:
indexref = msmarco_dataset.get_index('terrier_unstemmed')
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 3213835
Number of terms: 16168096
Number of postings: 905088837
Number of fields: 0
Number of tokens: 2204592607
Field names: []
Positions:   false



In [ ]:
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2", "WMODEL:BM25", "WMODEL:TF_IDF"], verbose= True)

In [ ]:
train_topics = msmarco_dataset.get_topics("train")
validation_topics = msmarco_dataset.get_topics("dev")
train_qrels = msmarco_dataset.get_qrels("train")
validation_qrels = msmarco_dataset.get_qrels("dev")

In [ ]:
lmart = lgb.LGBMRanker(task="train",
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=100,
    max_bin=255,
    num_leaves=7,
    objective="lambdarank",
    metric=["ndcg", 'map'],
    learning_rate= 0.1,
    ndcg_eval_at=[1, 3, 5, 10],
    importance_type="gain",
    num_iterations=5)

lmart_pipe = pipeline >> pt.ltr.apply_learned_model(lmart, form="ltr")
lmart_pipe.fit(train_topics[:500], train_qrels, validation_topics, validation_qrels)

FBR(BM25 and 4 features):   0%|          | 0/500 [00:00<?, ?q/s]

FBR(BM25 and 4 features):   0%|          | 0/5193 [00:00<?, ?q/s]

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's ndcg@1: 0.297766	valid_0's map@1: 0.297766
[2]	valid_0's ndcg@1: 0.295262	valid_0's map@1: 0.295262
[3]	valid_0's ndcg@1: 0.297381	valid_0's map@1: 0.297381
[4]	valid_0's ndcg@1: 0.293143	valid_0's map@1: 0.293143
[5]	valid_0's ndcg@1: 0.296418	valid_0's map@1: 0.296418


In [ ]:
lmart_results_perquery = pt.pipelines.Experiment([lmart_pipe],
                                msmarco_dataset.get_topics("test"),
                                msmarco_dataset.get_qrels("test"), ["recip_rank", "ndcg","map"],
                                names=["LambdaMART"],
                                perquery = True)


FBR(BM25 and 4 features):   0%|          | 0/200 [00:00<?, ?q/s]

/usr/local/lib/python3.7/dist-packages/pyterrier/pipelines.py:108: UserWarning: 157 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.')


In [ ]:
print(lmart_results_perquery[~lmart_results_perquery['value'].isnull()])

          name      qid     measure     value
24  LambdaMART  1037798         map  0.016614
25  LambdaMART  1037798  recip_rank  0.500000
26  LambdaMART  1037798        ndcg  0.117215
60  LambdaMART   104861         map  0.079013
61  LambdaMART   104861  recip_rank  0.500000
..         ...      ...         ...       ...
28  LambdaMART   915593  recip_rank  1.000000
29  LambdaMART   915593        ndcg  0.593778
36  LambdaMART   962179         map  0.175655
37  LambdaMART   962179  recip_rank  0.125000
38  LambdaMART   962179        ndcg  0.487800

[129 rows x 4 columns]


In [ ]:
lmart_results = pt.pipelines.Experiment([lmart_pipe],
                                msmarco_dataset.get_topics("test"),
                                msmarco_dataset.get_qrels("test"), ["recip_rank", "ndcg","map"],
                                names=["LambdaMART"],
                                perquery = False)

FBR(BM25 and 4 features):   0%|          | 0/200 [00:00<?, ?q/s]

In [ ]:
print(lmart_results)

         name  recip_rank      ndcg      map
0  LambdaMART    0.486709  0.328899  0.15441
